# Importing Library

In [ ]:
! pip install config

In [ ]:
! pip install folium

In [1]:
import pandas as pd
import numpy as np
import requests
import folium
import config
from math import radians, cos, sin, asin, sqrt

# Data Sourcing

## Accidents Data Loading and Preprocessing

In [2]:
data2020 = pd.read_csv('../raw_data/B_Accident_data/AfSBBB_BE_LOR_Strasse_Strassenverkehrsunfaelle_2020_Datensatz.csv', sep=';')
data2019 = pd.read_csv('../raw_data/B_Accident_data/AfSBBB_BE_LOR_Strasse_Strassenverkehrsunfaelle_2019_Datensatz.csv', sep=';')
data2018 = pd.read_csv('../raw_data/B_Accident_data/AfSBBB_BE_LOR_Strasse_Strassenverkehrsunfaelle_2018_Datensatz.csv', sep=';')
df = pd.concat([data2018, data2019, data2020])
df

,OBJECTID,LAND,BEZ,LOR,STRASSE,LOR_ab_2021,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,...,IstKrad,IstGkfz,IstSonstig,STRZUSTAND,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84,USTRZUSTAND,"YGCSWGS84,,,,"
0,112695,11,2,2050602.0,Samariterviertel,2500729.0,2018,1,15,4,...,0,0,0,1.0,"803604,1562","5827580,738","13,4750178","52,51359681",NaN,NaN
1,112705,11,12,12304314.0,Ziekowstraße/Freie Scholle,12500824.0,2018,1,11,2,...,0,0,0,0.0,"790639,5854","5835009,589","13,29102205","52,58725906",NaN,NaN
2,112726,11,2,2040501.0,Barnimkiez,2400520.0,2018,1,9,3,...,0,0,0,0.0,"799827,9618","5828733,523","13,42057818","52,52601854",NaN,NaN
3,112737,11,7,7020202.0,Volkspark (Rudolf-Wilde-Park),7200308.0,2018,1,17,2,...,0,0,0,0.0,"795223,509","5823525,402","13,34828776","52,48184447",NaN,NaN
4,112747,11,3,3020209.0,Niederschönhausen,3200206.0,2018,1,15,4,...,0,0,0,1.0,"798261,3849","5835047,26","13,40322797","52,58347154",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11805,146061,11,7,7040403.0,NaN,7400823.0,2020,12,7,4,...,0,0,0,NaN,"797339,7447","5821935,194","13,37794126",NaN,1,"52,46644169"
11806,146062,11,12,12231101.0,NaN,12400722.0,2020,12,17,3,...,0,0,0,NaN,"791814,8333","5837953,171","13,31091476",NaN,1,"52,61301792"
11807,146065,11,1,1011306.0,NaN,1100313.0,2020,12,14,3,...,0,0,1,NaN,"799900,8048","5826514,725","13,41964879",NaN,0,"52,50609051"
11808,146068,11,1,1011204.0,NaN,1100207.0,2020,12,16,3,...,0,0,0,NaN,"797982,8966","5826897,222","13,39181741",NaN,1,"52,51057"


In [3]:
df.keys()

Index(['OBJECTID', 'LAND', 'BEZ', 'LOR', 'STRASSE', 'LOR_ab_2021', 'UJAHR',
       'UMONAT', 'USTUNDE', 'UWOCHENTAG', 'UKATEGORIE', 'UART', 'UTYP1',
       'ULICHTVERH', 'IstRad', 'IstPKW', 'IstFuss', 'IstKrad', 'IstGkfz',
       'IstSonstig', 'STRZUSTAND', 'LINREFX', 'LINREFY', 'XGCSWGS84',
       'YGCSWGS84', 'USTRZUSTAND', 'YGCSWGS84,,,,'],
      dtype='object')

In [ ]:
#Preprocessing
df['XGCSWGS84'] = df['XGCSWGS84'].map(lambda x: x.replace(',','.')).astype(float)
df['YGCSWGS84'] = df['YGCSWGS84'].map(lambda x: x.replace(',','.')).astype(float)

In [ ]:
#1 == Unfall mit Fahrradbeteiligung
df.IstRad.value_counts()

In [ ]:
bike_accidents = pd.DataFrame(df.groupby('UJAHR').IstRad.value_counts())
bike_accidents

In [ ]:
#Optional for look at individual years

#accidents = {
#    'w/ bicycle': [
#    data2018.IstRad.value_counts()[1],
#    data2019.IstRad.value_counts()[1],
#    data2020.IstRad.value_counts()[1]
#    ],
#    'w/o bicycle': [
#    data2018.IstRad.value_counts()[0], 
#    data2019.IstRad.value_counts()[0], 
#    data2020.IstRad.value_counts()[0]
#    ],
#    'involvement (%)': [
#    round(data2018.IstRad.value_counts()[1]/len(data2018)*100, 2),
#    round(data2019.IstRad.value_counts()[1]/len(data2019)*100, 2),
#    round(data2020.IstRad.value_counts()[1]/len(data2020)*100, 2)   
#    ],
#}
#
#df = pd.DataFrame(accidents, index = [2018, 2019, 2020])
#df

## infraVelo API

In [ ]:
url = 'https://www.infravelo.de/api/v1/projects/'
url2 = 'https://www.infravelo.de/api/v1/projects/50/50/'
url3 = 'https://www.infravelo.de/api/v1/projects/100/50/'
url4 = 'https://www.infravelo.de/api/v1/projects/150/50/'
response = requests.get(url3).json()
response

In [ ]:
names=[]
for i in range(len(response['results'])):
    names.append(response['results'][i]['title'])
pd.DataFrame(names)

In [ ]:
#Reading APIs

#for i in range(1,50):
#    print(i, response['results'][i]['title'])

In [ ]:
#KML data stlip / Wisbyer Strasse
kml_str = response['results'][41]['kml'].split()
kml_str

In [ ]:
kml_str[44]

In [ ]:
#Organize KML data per block (only the first one)
coordinate1, coordinate2, coordinate3, coordinate4, coordinate5 = [],[],[],[],[]
for i in range(len(kml_str)):
    if kml_str[i] == '<coordinates>':
        coordinate1.append(kml_str[i+1])
#    if kml_str[i] != '</coordinates>':

print(coordinate1)

In [ ]:
#Manually took the coordinates for test
test_location = [13.4405757994,52.4737692995,
 13.4405113004,52.4738815993,
 13.4404971005,52.4739062994,
 13.4404219001,52.474037199,
 13.4403887007,52.4740936991,
 13.4403685999,52.4741284993,
 13.4403181006,52.4742146991,
 13.4402538,52.4743250988,
 13.4400138998,52.4747357993,
 13.4399429006,52.4748919993,
 13.4398426995,52.4751500993,
 13.4398239993,52.4751980988,
 13.4397882993,52.475289899]
test_tup = tuple(test_location)
test_location = np.array(test_location).reshape(13,2)
test_location[:,0], test_location[:,1] = test_location[:,1], test_location[:,0].copy()

In [ ]:
points = tuple(test_location)
points

In [ ]:
test_location2 = [13.4495974004,52.537157899,
 13.4496928005,52.5370618994,
 13.4497330002,52.537021499,
 13.4500717999,52.536680499,
 13.4501126004,52.5366449993,
 13.4502748004,52.5364561987,
 13.4506326007,52.5361042989,
 13.4509465006,52.5358140995,
 13.4510125007,52.535746399,
 13.4514056996,52.5353429994,
 13.4515473999,52.5351961991,
 13.4517637993,52.5349717992,
 13.4517834005,52.5349520992,
 13.4518773006,52.5348575987,
 13.4520993995,52.5346339991,
 13.4524955995,52.5342245987,
 13.4529111006,52.5337948994,
 13.4532731002,52.5334266992,
 13.4533520007,52.533346299,
 13.4533856006,52.5333112992,
 13.4536300996,52.5330603989,
 13.4537551993,52.5329256994,
 13.4538386007,52.5328358991,
 13.4542387,52.532435899,
13.4552262993,52.5314222992,
13.4556814993,52.5309736987,
13.4557106998,52.5309448995,
13.4558564007,52.5308011988,
13.4559387993,52.5307130994,
13.4559658004,52.5306841995,
13.4562414006,52.530389499,
13.4564922995,52.5301492992,
13.4566411002,52.5300068994,
13.4567235004,52.5299279991,
13.4567877996,52.5298664994,
13.4571880004,52.5294601988,
13.4574770006,52.5291722992,
13.4575215004,52.5291279989,
13.4576247996,52.5290250992]
test_tup2 = tuple(test_location2)
test_location2 = np.array(test_location2).reshape(39,2)
test_location2[:,0], test_location2[:,1] = test_location2[:,1], test_location2[:,0].copy()
points2 = tuple(test_location2)

In [ ]:
test_location3 = [13.4301194998,52.5513524989,
 13.4299060004,52.5514397991,
 13.4288630002,52.5518523995,
 13.4280413005,52.5521647987,
 13.4274770995,52.5523852991,
 13.4274448004,52.5523978988,
 13.4273235004,52.5524273988,
 13.4271935996,52.552457099,
 13.4269072006,52.5525224995,
 13.4265339005,52.5525745992,
 13.4256882003,52.552666899,
 13.4255319997,52.5526838994,
 13.4253715994,52.5527013993,
 13.4253189993,52.5527070994,
 13.4245639994,52.5527907994,
 13.4240729997,52.5528430995,
 13.4238913993,52.5528651991,
 13.4233038,52.5529282991,
 13.4224540995,52.5530204992,
 13.4220908999,52.5530595989,
 13.4208025006,52.5532058993,
 13.4199398998,52.5533038991,
 13.4196113997,52.5533412993,
 13.4184403998,52.5534555987,
 13.4159971003,52.5537321993,
 13.4157511004,52.5537588993,
 13.4154117007,52.5537956987,
 13.4149393995,52.5538518994,
 13.4148765001,52.5538578993,
 13.4147283003,52.5538674995]
test_tup3 = tuple(test_location3)
test_location3 = np.array(test_location3).reshape(30,2)
test_location3[:,0], test_location3[:,1] = test_location3[:,1], test_location3[:,0].copy()
points3 = tuple(test_location2)

## Map

In [ ]:
#Prepare accidents data around here
df = df[df.IstRad == 1]
df.head()

In [ ]:
#Cutting df only for the test area. 
df1 = df[df.XGCSWGS84 >= 13.43]
df2 = df1[df1.XGCSWGS84 <= 13.45]
df3 = df2[df2.YGCSWGS84 >= 52.47]
df4 = df3[df3.YGCSWGS84 <= 52.48]
test_accidents = tuple(np.array([df4.YGCSWGS84, df4.XGCSWGS84]).T.tolist())

In [ ]:
#General accidents data per year
accidents_2018 = tuple(np.array([df[df.UJAHR == 2018].YGCSWGS84, df[df.UJAHR == 2018].XGCSWGS84]).T.tolist())
accidents_2019 = tuple(np.array([df[df.UJAHR == 2019].YGCSWGS84, df[df.UJAHR == 2019].XGCSWGS84]).T.tolist())
accidents_2020 = tuple(np.array([df[df.UJAHR == 2020].YGCSWGS84, df[df.UJAHR == 2020].XGCSWGS84]).T.tolist())

In [ ]:
#Plotting

#accidents = test_accidents

m = folium.Map(location=[52.52000, 13.4050],tiles='Stamen Toner', zoom_start=15)
#folium.PolyLine(locations=, color='green', weight=10, opacity=1).add_to(m)
#markers=[(52.475,13.438),(52.476, 13.441),(52.473,13.439),(52.473,13.442)]
for each in accidents_2018: # 11 accidents
    folium.CircleMarker(each, radius=2, color='blue').add_to(m)
#for each in accidents_2019: # 9 accidents
    #folium.CircleMarker(each, radius=2, color='yellow').add_to(m)
#for each in accidents_2020: # 6 accidents
   # folium.CircleMarker(each, radius=2,color='red').add_to(m)
#    folium.Marker(each).add_to(m) # Very Heavy!!
m

In [ ]:
#coordinates = [48.8649224, 2.3800903] # 16 Villa Gaudelet, Paris

#m = folium.Map(location=coordinates, zoom_start=8)
#folium.Marker(coordinates, tooltip='Le Wagon Paris').add_to(m)
#folium.Circle(coordinates, radius=100 * 1000).add_to(m)
#m

In [ ]:
#def geocode(address):
#    params = { "q": address, 'format': 'json' }
#    places = requests.get(f"https://nominatim.openstreetmap.org/search", params=params).json()
#    return [places[0]['lat'], places[0]['lon']]
#    
#def draw_100km_map(address):
#    coordinates = geocode(address)
#    m = folium.Map(location=coordinates, zoom_start=8)
#    folium.Marker(coordinates, tooltip='Le Wagon Paris').add_to(m)
#    folium.Circle(coordinates, radius=100 * 1000).add_to(m)
#    return m

In [ ]:
#address = input()
#draw_100km_map(address)

In [ ]:
#EXAMPLE OF A HEATMAP
#from folium.plugins import HeatMap

### Create a Map instance
#mp = folium.Map(location=[52.5, 13.5], tiles = 'stamentoner', zoom_start=9, control_scale=True)

### Add heatmap to map instance
#### Available parameters: HeatMap(data, name=None, min_opacity=0.5, max_zoom=18, max_val=1.0, radius=25, blur=15, gradient=None, overlay=True, control=True, show=True)
#HeatMap(points3).add_to(mp)

### Alternative syntax:
###m.add_child(HeatMap(points_array, radius=15))

#### Show map
#mp

## Calculate Distance from coordinates

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in kilometers. Use 3956 for miles
    return c * r

## EDA

In [ ]:
df.keys()

In [ ]:
#dropping unnecessary columns
df = df.drop(columns=['IstRad', 'LAND'])

In [ ]:
df

In [ ]:
df.info()

In [ ]:
#Missing values
df.isnull().sum()

In [ ]:
#NaN percentage for each column
df.isnull().sum().sort_values(ascending=False)/len(df) 

In [ ]:
#counting duplicates
df.duplicated().sum()

In [ ]:
#df['USTRZUSTAND'] = cars['USTRZUSTAND'].map({"USTRZUSTAND":4,
                                                    "six":6,
                                                    "five":5,
                                                    "eight":8,
                                                    "two":2,
                                                    "three":3,
                                                    "twelve":12})